# Chap 10 데이터 집계와 그룹 연산

 
* 하나 이상의 를 이용해서 pandas 객체를 여러 조각으로 나누는 법

* 합계, 평균, 표준편차, 사용자 정의 함수 같은 그룹 요약 통계 계산법

* 정규화, 선형회귀, 등급 또는 부분 집합 선택 등 집단 내 변형이나 다른 조작을 적용하는 방법

* 피벗테이블과 교차일람표를 구하는 방법

* 변위치 분석과 다른 통계 집단 분석을 수행하는 방법


## GroupBy 메카닉 


1. 분리 
 - 하나의 Key를 기준으로 **분리**
 
2. 적용
 - 함수를 나뉘어진 그룹에 **적용**시켜 새로운 값 얻어내기

3.  결합
 - 함수가 적용된 결과를 하나의 객체로 **결합**
 
 
---------------------------------

Google의 논문 [MapReduce: Simplified Data Processing on Large Clusters](https://research.google/pubs/pub62/)에 영향받은

데이터 분석에서는 Hadley Wickhan의 논문 [The Split-Apply-Combine Strategy for Data Analysis](https://www.jstatsoft.org/v40/i01/paper)

에서 시작되었음
 
------------------------------ 
> 두 논문 다 시간 날 때 읽어 보시길 권합니다..!

In [4]:
import numpy as np
import pandas as pd

In [5]:
df = pd.DataFrame(
    {
        "key1": ["a", "a", "b", "b", "a"],
        "key2": ["one", "two", "one", "two", np.nan],
        "data1": np.random.randn(5),
        "data2": np.random.randn(5),
    }
)
df

,key1,key2,data1,data2
0,a,one,-0.020457,0.029252
1,a,two,0.102137,-1.332266
2,b,one,-0.533152,0.680162
3,b,two,-0.690446,-1.299890
4,a,NaN,0.690403,1.776467


In [6]:
grouped = df["data1"].groupby(df["key1"])
grouped  # 단순히 groupby만 쓰면 groupby 객체만 나옵니다.

In [7]:
grouped.mean()  # key1 칼럼 기준으로 묶어서 평균을 계산해주었음.
# 새로운 이 Series 객체의 색인은 우리가 groupby로 지정한 'key1임'

key1
a    0.257361
b   -0.611799
Name: data1, dtype: float64

In [8]:
means = (
    df["data1"].groupby([df["key1"], df["key2"]]).mean()
)  # groupby key를 key1, key2 두개를 줬기에 계층적 index를 갖게됨
means

key1  key2
a     one    -0.020457
      two     0.102137
b     one    -0.533152
      two    -0.690446
Name: data1, dtype: float64

In [9]:
means.unstack()  # unstack으로 풀어버리기도 가능

key2,one,two
key1,,
a,-0.020457,0.102137
b,-0.533152,-0.690446


In [10]:
states = np.array(["Ohio", "California", "California", "Ohio", "Ohio"])
years = np.array([2005, 2005, 2006, 2005, 2006])
df["data1"].groupby([states, years]).mean()  # 기존 data1에 없던 index를 넣어서도 가능함.

California  2005    0.102137
            2006   -0.533152
Ohio        2005   -0.355452
            2006    0.690403
Name: data1, dtype: float64

In [11]:
df.groupby(["key1", "key2"]).mean()

data1     data2
key1 key2                    
a    one  -0.020457  0.029252
     two   0.102137 -1.332266
b    one  -0.533152  0.680162
     two  -0.690446 -1.299890

In [12]:
df.groupby(
    "key1"
).mean()  # 위와 비교하면 key2가 빠져있는데 key2는 숫자가 아니라 (범주형 데이터라) mean 계산이 불가능하기 때문
# 이런 칼럼을 성가신 칼럼 nuisance column이라 부르고 결과에서 제외함.

,data1,data2
key1,,
a,0.257361,0.157818
b,-0.611799,-0.309864


In [13]:
df.groupby(["key1", "key2"]).size()  # 이때 nan은 제외힘

key1  key2
a     one     1
      two     1
b     one     1
      two     1
dtype: int64

### 그룹간 순회하기

In [14]:
for name, group in df.groupby("key1"): # 하나씩 for 문으로 받기도 가능. 뭔가 각 종류별로 데이터를 넣을 때 사용할 수 있을 듯 `
    print(name)
    print('-'* 10)
    print(group)       
    print('\n')

a
----------
  key1 key2     data1     data2
0    a  one -0.020457  0.029252
1    a  two  0.102137 -1.332266
4    a  NaN  0.690403  1.776467


b
----------
  key1 key2     data1     data2
2    b  one -0.533152  0.680162
3    b  two -0.690446 -1.299890




In [15]:
for (k1, k2), group in df.groupby(['key1', 'key2']):
    print((k1, k2))
    print('-' * 10)
    print(group)
    print('\n')

('a', 'one')
----------
  key1 key2     data1     data2
0    a  one -0.020457  0.029252


('a', 'two')
----------
  key1 key2     data1     data2
1    a  two  0.102137 -1.332266


('b', 'one')
----------
  key1 key2     data1     data2
2    b  one -0.533152  0.680162


('b', 'two')
----------
  key1 key2     data1    data2
3    b  two -0.690446 -1.29989




In [16]:
pieces = dict(list(df.groupby('key1')))   # dic 형식으로 받기 도 가능 
pieces             

{'a':   key1 key2     data1     data2
 0    a  one -0.020457  0.029252
 1    a  two  0.102137 -1.332266
 4    a  NaN  0.690403  1.776467, 'b':   key1 key2     data1     data2
 2    b  one -0.533152  0.680162
 3    b  two -0.690446 -1.299890}

In [17]:
df.dtypes

key1      object
key2      object
data1    float64
data2    float64
dtype: object

In [18]:
grouped = df.groupby(df.dtypes, axis=1)  # 행을 기준이 아닌 열(axis = 1)을 기준으로도 그룹을 만들 수 있음 
for dtype, group in grouped:
    print(dtype)
    print('-'* 20)
    print(group)
    print('\n')

float64
--------------------
      data1     data2
0 -0.020457  0.029252
1  0.102137 -1.332266
2 -0.533152  0.680162
3 -0.690446 -1.299890
4  0.690403  1.776467


object
--------------------
  key1 key2
0    a  one
1    a  two
2    b  one
3    b  two
4    a  NaN




### 컬럼이나 컬럼의 일부만 선택하기 
생략

### dict과 Series에서 그룹핑하기

생략

### 함수로 그루핑하기 

In [23]:
people = pd.DataFrame(np.random.randn(5, 5),
                      columns=['a', 'b', 'c', 'd', 'e'],
                      index=['Joe', 'Steve', 'Wes', 'Jim', 'Travis'])
people

,a,b,c,d,e
Joe,-1.895139,0.877092,-0.429203,0.228393,1.001206
Steve,-0.811868,-0.604670,0.581216,-0.810263,1.353533
Wes,-0.054384,-0.619279,-0.463613,-0.874613,-0.114991
Jim,1.124818,0.664126,1.388410,-0.776159,-0.826000
Travis,-0.409048,1.262777,0.361698,-0.227387,-0.279755


In [24]:
people.groupby(len).sum() # axis = 0에 대해서 len()을 적용해서 3(Joe, Wes, Jim), 5(Steve),6(Travis) 순으로 정렬함 


,a,b,c,d,e
3,-0.824705,0.921940,0.495594,-1.422380,0.060216
5,-0.811868,-0.604670,0.581216,-0.810263,1.353533
6,-0.409048,1.262777,0.361698,-0.227387,-0.279755


In [25]:
key_list = ['one', 'one', 'one', 'two', 'two']
people.groupby([len, key_list]).min()  # len()의 결과와 key_list 값들 을 기준으로 multi index로 정렬함 

a         b         c         d         e
3 one -1.895139 -0.619279 -0.463613 -0.874613 -0.114991
  two  1.124818  0.664126  1.388410 -0.776159 -0.826000
5 one -0.811868 -0.604670  0.581216 -0.810263  1.353533
6 two -0.409048  1.262777  0.361698 -0.227387 -0.279755

### 인덱스 단계로 그루핑
생략

## 데이터 집계

자신만의 데이터 함수 사용하려면 agg메서드에 해당 함수를 넘기면 된다.

In [30]:
df
grouped = df.groupby('key1')

In [29]:
def peak_to_peak(arr):
    return arr.max() - arr.min()
grouped.agg(peak_to_peak)

,data1,data2
key1,,
a,0.710860,3.108733
b,0.157295,1.980052


### 컬럼에 여러 가지 함수 적용하기
생략